# AutoML 알고리즘

---------------
## 0. 환경설정

In [ ]:
##### 라이브러리 호출 #####
import numpy as np
import pandas as pd
import time
import glob
import pickle
import itertools

import h2o
from h2o.automl import H2OAutoML
from h2o.estimators.gbm import H2OGradientBoostingEstimator
from sklearn.model_selection import train_test_split
from statsmodels.formula.api import ols

import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib import font_manager, rc
%matplotlib inline

import warnings
warnings.filterwarnings('ignore')

# 데이터프레임 출력 옵션
pd.set_option('display.max_columns', 100)

#지수표현
pd.options.display.float_format = '{:.5f}'.format

---------------
## 1. 입력값 기입

In [ ]:
# data_folder : 원본데이터 위치(폴더명)
# save_folder : 결과 저장 위치(폴더명)
# file_nm : 파일명
# target : target 컬럼들 (하나의 'target'명으로 병합할 순서대로, 컬럼명을 리스트 형식으로 기입)
# y_colnm : y 컬럼명
# year_colnm : 연도 컬럼명
# test_year : test년도

In [ ]:
## 파일명 / 해당 파일의 target 컬럼들 (target명으로 합치고 싶은 순서대로 기입) / 파일마다의 y 컬럼명 / 연도 컬럼명 / test년도 입력
data_folder = 'data'
save_folder = 'result'
file_nm = ['merge_age_new','merge_cls_new','merge_dis_new','merge_part_age','merge_part_dis9']
target = [['SIDO','SCHOOL_TP'],['SIDO'],['SIDO','DIS_TYPE'],['SIDO_EDU_NM','PART_EDU_NM','PART_EDU_CD','SCHOOL_TP'],['PART_EDU_NM','PART_EDU_CD']]
y_colnm = ['SEP_CNT','SEP_CNT','SEP_CNT','SEP_CNT','SEP_CNT']
year_colnm = ['BASE_YY','BASE_YY','BASE_YY','BASE_YY','BASE_YY']
test_year = ['2021','2021','2021','2021','2021']

In [ ]:
## 독립변수(x)에 대하여 표준화 진행 시 원하는 표준화 기법 선택 (입력 안하면 표준화 진행 X)
# 1 : 표준화1(StandardScaler) : 평균 = 0 / 표준편차 = 1
# 2 : 표준화2(Normalization) : MinMaxScaler : 최소값 0 ~ 최대값 1 
# 3 : 표준화3(RobustScaler) : 중앙값 = 0 / IQR(1분위(25%) ~ 3분위(75%)) = 1 
select_scaler = 1  # (1, 2, 3) 중 택 1

---------------
## 2. 모델 생성

In [ ]:
total_start_time = time.time()

In [ ]:
# 파일 정보 통합
file_info = pd.DataFrame(file_nm).rename(columns = {0:'file_nm'})
file_info['target'] = target
file_info['y_colnm'] = y_colnm
file_info['year_colnm'] = year_colnm
file_info['test_year'] = test_year

# h2o 호출
h2o.init(nthreads=1)

for file_num in range(len(file_info)):

    # ------------------------------------------------------------------------------------------------------------------ #
    # 분석 정보 호출
    file_nm = file_info['file_nm'][file_num]
    target = file_info['target'][file_num]
    y_colnm = file_info['y_colnm'][file_num]
    year_colnm = file_info['year_colnm'][file_num]
    test_year = file_info['test_year'][file_num]

    # 데이터 호출
    tot_data = pd.read_csv(data_folder + '/' + file_nm + '.csv', dtype='str', encoding = 'cp949')
    tot_data.rename(columns = {year_colnm:'YEAR'}, inplace=True)  # 연도 컬럼명 변경
    
    # train 연도 정의 (위에서 정의한 test년도 제외)
    train_year = list(tot_data['YEAR'].unique())
    train_year.remove(test_year)
    # ------------------------------------------------------------------------------------------------------------------ #
    # target 리스트를 병합하여 하나의 컬럼('target')으로 생성 (ex. 강원,특수교육 => 강원_특수교육)
    for trg_num in range(len(target)):
        if len(target) == 1:
            tot_data['target'] = tot_data[target[trg_num]]
        else:
            if trg_num == 0:
                tot_data['target'] = tot_data[target[trg_num]]
            else:
                tot_data['target'] = tot_data['target'] + '_' + tot_data[target[trg_num]]
    # ------------------------------------------------------------------------------------------------------------------ #
    ## 데이터 구분
    # 월 데이터
    if file_nm.find('month') != -1:
        tot_data['STAND_TIME'] = tot_data['YEAR'] + tot_data['MONTH']
        except_colnm = ['YEAR','MONTH','STAND_TIME']
    # 분기 데이터
    elif file_nm.find('quarter') != -1:
        tot_data['STAND_TIME'] = tot_data['YEAR'] + tot_data['QUARTER']
        except_colnm = ['YEAR','QUARTER','STAND_TIME']
    # 반기 데이터
    elif file_nm.find('half') != -1:
        tot_data['STAND_TIME'] = tot_data['YEAR'] + tot_data['HALF']
        except_colnm = ['YEAR','HALF','STAND_TIME']
    # 연 데이터
    else:
        tot_data['STAND_TIME'] = tot_data['YEAR']  # YEAR 컬럼 생성
        except_colnm = ['YEAR','STAND_TIME']
    # ------------------------------------------------------------------------------------------------------------------ #
    # 독립변수 컬럼명 정의 : x_colnm
    tot_colnm = list(tot_data.columns)                                 # 전체 컬럼명
    except_colnm = ([y_colnm] + target + ['target'] + except_colnm)    # 독립변수에서 제외할 컬럼명
    x_colnm = list(set(tot_colnm).difference(set(except_colnm)))       # 독립변수 컬럼명

    # tot_data 재정의 : 필요 컬럼만 추출
    tot_data = tot_data[['STAND_TIME','YEAR','target'] + [y_colnm] + x_colnm]
    # ------------------------------------------------------------------------------------------------------------------ #
    ## 모델 생성 start
    
    # loop_target : 타겟 리스트 : 소단위 모델 생성
    loop_target = list(tot_data['target'])  
    for trg in loop_target:

        print(trg)

        # -------------------------------------------------------------------------------------------------------------- #
        # 분석 수행 데이터 정의
        data = tot_data.loc[tot_data['target'] == trg,].sort_values(by = 'STAND_TIME').reset_index(drop=True)
        # -------------------------------------------------------------------------------------------------------------- #
        # 데이터 형 변환(str -> float)
        for chg_col in ([y_colnm] + x_colnm):
            data[chg_col] = data[chg_col].astype('float')
        # -------------------------------------------------------------------------------------------------------------- #
        # (train / test) 데이터 분할
        train = data.loc[data['YEAR'].isin(train_year),[y_colnm] + x_colnm]
        test = data.loc[~data['YEAR'].isin(train_year),[y_colnm] + x_colnm]

        # (train_x / train_y / test_x / test_y) 데이터 분할
        train_x = train[x_colnm].reset_index(drop=True)
        train_y = train[[y_colnm]].reset_index(drop=True)
        test_x = test[x_colnm].reset_index(drop=True)
        test_y = test[[y_colnm]].reset_index(drop=True)
        # -------------------------------------------------------------------------------------------------------------- #
        ## 표준화 수행
        
        # StandardScaler : 평균 0, 표준편차 1
        if select_scaler == 1:
            from sklearn.preprocessing import StandardScaler
            scaler = StandardScaler()   
            mody_train_x = pd.DataFrame(scaler.fit_transform(train_x), columns = list(train_x.columns))
            mody_test_x = pd.DataFrame(scaler.transform(test_x), columns = list(test_x.columns))
        
        # Normalization : MinMaxScaler : 최소값 0 ~ 최대값 1
        elif select_scaler == 2:
            from sklearn.preprocessing import MinMaxScaler
            scaler = MinMaxScaler()
            mody_train_x = pd.DataFrame(scaler.fit_transform(train_x), columns = list(train_x.columns))
            mody_test_x = pd.DataFrame(scaler.transform(test_x), columns = list(test_x.columns))

        # RobustScaler : 중앙값 0, IQR(1분위(25%) ~ 3분위(75%)) 1 : 이상치(outlier) 영향 최소화 / 더 넓게 분포
        elif select_scaler == 3:
            from sklearn.preprocessing import RobustScaler
            scaler = RobustScaler()
            mody_train_x = pd.DataFrame(scaler.fit_transform(train_x), columns = list(train_x.columns))
            mody_test_x = pd.DataFrame(scaler.transform(test_x), columns = list(test_x.columns))
        
        else:
            mody_train_x = train_x
            mody_test_x = test_x
        # -------------------------------------------------------------------------------------------------------------- #
        # 모델에 사용할 train, test 데이터셋
        mdl_train_data = pd.concat([train_y, mody_train_x], axis = 1)
        mdl_test_data = mody_test_x
        # -------------------------------------------------------------------------------------------------------------- #
        ## 독립변수 선택

        # 1. 상관분석 : 상관분석은 train 데이터만 이용 (test 데이터 이용 X)
        corr_data = pd.concat([train_y,mody_train_x], axis = 1)
        corr = corr_data.corr(method = 'pearson')  # default는 'pearson'
        corr = corr.reset_index().rename(columns = {'index':'COLNM'})
        corr = corr.loc[corr['COLNM'] != y_colnm,]
        corr = corr[corr[y_colnm] >= 0.5]

        # 2. 상관분석 결과로 선택된 독립변수 목록
        mdl_x_colnm = list(corr['COLNM'])

        # 3. 독립변수 목록 중 null값이 없는 독립변수만 선택
        train_na_col = []
        for col in mdl_test_data.columns:
            if len(mdl_train_data.loc[mdl_train_data[col].isna(),]) != 0:
                train_na_col.append(col)

        test_na_col = []
        for col in mdl_test_data.columns:
            if len(mdl_test_data.loc[mdl_test_data[col].isna(),]) != 0:
                test_na_col.append(col)

        tot_na_col = list(set(train_na_col + test_na_col))  # null값이 있는 독립변수들
        mdl_x_colnm = list(set(mdl_x_colnm).difference(set(tot_na_col)))  # 모델에 사용할 독립변수들
        # -------------------------------------------------------------------------------------------------------------- #
        # h2o 데이터프레임 형식으로 변환
        h2o_train_data = h2o.H2OFrame(mdl_train_data)
        h2o_test_data = h2o.H2OFrame(mdl_test_data)

        ## 모델 생성
        model = H2OAutoML(max_models=20, max_runtime_secs=10, seed=1234)
        model.train(x = mdl_x_colnm, y = y_colnm,
                    training_frame = h2o_train_data)  # x : 독립변수 / y : 종속변수 / training_frame : 학습데이터 / 모델 검증은 pass
        # -------------------------------------------------------------------------------------------------------------- #
        # # View the AutoML Leaderboard
        # lb = model.leaderboard
        # lb.head(rows = 10)  # 가장 성능 좋은 모델 top 10개 확인
        # model.leader  # 리더보드 값 확인 : The leader model is stored here

        # ## 모델 조사
        # m = model.leader  # Get the best model using the metric
        # m = model.get_best_model()  # this is equivalent to

        ## AutoML 출력
        # Get leaderboard with all possible columns
        lb = h2o.automl.get_leaderboard(model, extra_columns = "ALL")  # lb : top 10개 모델에 대한 리더보드 확인
        save_lb = lb.as_data_frame()  # pandas 데이터프레임으로 형변환
        # -------------------------------------------------------------------------------------------------------------- #
        ## 예측 수행
        pred = model.predict(h2o_test_data)

        ## h2o 데이터프레임을 pandas 데이터프레임으로 변환
        pred = h2o.as_list(pred, use_pandas=True)  # 또는 pred.as_data_frame()
        pred.rename(columns={'predict':'PREDICT'}, inplace=True)
        # -------------------------------------------------------------------------------------------------------------- #
#         ## h2o 종료
#         h2o.cluster().shutdown()
        # -------------------------------------------------------------------------------------------------------------- #
        ## 결과값 정리
        rslt = pd.concat([pred, test_y], axis = 1)
        rslt['target'] = trg
        rslt['stand_time'] = list(data.loc[~data['YEAR'].isin(train_year),'STAND_TIME'])
        rslt['BEST_MDL'] = save_lb['model_id'][0]
        rslt['DIFF'] = rslt['PREDICT'] - rslt['SEP_CNT']
        rslt['mdl_x_colnm'] = str(mdl_x_colnm)

        rslt['PREDICT'] = round(rslt['PREDICT'],4)
        rslt['SEP_CNT'] = round(rslt['SEP_CNT'],4)
        rslt['DIFF'] = round(rslt['DIFF'],4)
        
        rslt = rslt[['target', 'stand_time', 'PREDICT', 'SEP_CNT', 'DIFF', 'BEST_MDL']]
        # -------------------------------------------------------------------------------------------------------------- #
        ## 결과값 저장
        if trg == loop_target[0]:
            fin_result = rslt
        else:
            fin_result = fin_result.append(rslt)

    fin_result.to_csv(save_folder + '/result_' + file_nm + '.csv', index=False, encoding = 'utf-8')

# h2o 종료
h2o.cluster().shutdown()

In [ ]:
print('총 모델 생성 시간 : ', time.time() - total_start_time)

------

In [ ]:
## AutoML 설명
# (1) AutoML은 총 20여 개의 머신러닝과 딥러닝 모델을 포함한 알고리즘임
# (2) 학습 시 해당 데이터에 가장 적합한 모델을 선택하며, 파라미터 또한 최적화 된 값을 선택해줌
# (3) AutoML은 머신러닝 기반의 모형이기 때문에, 통계적 모형인 회귀 모형과는 달리 다중공선성 등의 변수 선택 과정을 진행하지 않음

## AutoML 변수 선택 과정(모델 개발 시 적용)
# (1) 독립변수에 대해 표준화
# - StandardScaler : 평균은 0, 표준편차는 1로 변환
# - Normalization : 최소값을 0, 최대값을 1로 변환 (0~1 사이의 값으로 변환)
# - RobustScaler : 중앙값을 0, IQR(1분위수~3분위수)을 1로 변환 : 이상치의 영향 최소화
# (2) 상관분석
# - 상관계수의 기준을 0.5 이상으로 잡아서 최대한 적용되지 않는 독립변수의 수를 줄임
# (3) Null 값이 포함된 변수 제외
# - 독립변수 중 특정 연도의 값이 Null 값인 경우, 해당 변수를 독립변수에서 제외

## 결과 해석
# - AutoML을 이용하여 (월/분기/반기/연) 모형을 개발해 본 결과, (시도별*연령별 / 시도별*학급(학교)별 / 시도별*장애영역별) 모형 모두 MSE 값이 가장 작은 모델로 ‘연 모형’이 채택되었음
# - AutoML은 20여 개의 모델이 포함된 알고리즘이므로, 각 단위 별로 20여 개의 모델을 이용하여 (월/분기/반기/연) 모형을 테스트 해봤다고 할 수 있음
# - 따라서, 각 단위 별 연 MSE 값이 가장 낮은 모델로 ‘연 모델'이 가장 많이 선택되었으므로 해당 데이터를 이용한 모델링은 ‘연 모델’이 적합하다고 판단됨

--------